# 保存和加载模型

In [19]:
import tensorflow as tf
from tensorflow import keras as K
from tensorflow.keras import layers
import numpy as np

## 只保存权值

可以使用tf.keras.Model.save_weight和tf.keras.Model.load_weight保存和加载权重

In [2]:
model = tf.keras.Sequential([
    layers.Dense(64, activation = 'relu', input_shape = (32,)),
    layers.Dense(10, activation = 'softmax')
])

model.compile(optimizer = tf.train.AdamOptimizer(0.001), loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
model.save_weights('./weights/my_model')

Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.


In [4]:
model.load_weights('./weights/my_model')

默认情况下keras是以tensorflow checkpoint的格式存储权重的，也可以指定为HDF5格式

In [5]:
model.save_weights('./weights/my_model.h5', save_format = 'h5')

In [6]:
model.load_weights('./weights/my_model.h5')

## 只保存网络配置

只保存网络配置的时候不会保留权重，支持两种序列化格式：json和yaml

### json格式

In [7]:
json_string = model.to_json()

In [8]:
json_string

'{"class_name": "Sequential", "config": {"name": "sequential", "layers": [{"class_name": "Dense", "config": {"name": "dense", "trainable": true, "batch_input_shape": [null, 32], "dtype": "float32", "units": 64, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null, "dtype": "float32"}}, "bias_initializer": {"class_name": "Zeros", "config": {"dtype": "float32"}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_1", "trainable": true, "dtype": "float32", "units": 10, "activation": "softmax", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null, "dtype": "float32"}}, "bias_initializer": {"class_name": "Zeros", "config": {"dtype": "float32"}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": n

In [9]:
import json

In [10]:
import pprint

In [11]:
pprint.pprint(json.loads(json_string))

{'backend': 'tensorflow',
 'class_name': 'Sequential',
 'config': {'layers': [{'class_name': 'Dense',
                        'config': {'activation': 'relu',
                                   'activity_regularizer': None,
                                   'batch_input_shape': [None, 32],
                                   'bias_constraint': None,
                                   'bias_initializer': {'class_name': 'Zeros',
                                                        'config': {'dtype': 'float32'}},
                                   'bias_regularizer': None,
                                   'dtype': 'float32',
                                   'kernel_constraint': None,
                                   'kernel_initializer': {'class_name': 'GlorotUniform',
                                                          'config': {'dtype': 'float32',
                                                                     'seed': None}},
                                   'ker

In [12]:
frest_model = K.models.model_from_json(json_string)

### yaml格式

使用yaml格式需要安装pyyaml, `pip install pyyaml`

In [13]:
yaml_string = model.to_yaml()

In [15]:
yaml_string

'backend: tensorflow\nclass_name: Sequential\nconfig:\n  layers:\n  - class_name: Dense\n    config:\n      activation: relu\n      activity_regularizer: null\n      batch_input_shape: !!python/tuple\n      - null\n      - 32\n      bias_constraint: null\n      bias_initializer:\n        class_name: Zeros\n        config:\n          dtype: float32\n      bias_regularizer: null\n      dtype: float32\n      kernel_constraint: null\n      kernel_initializer:\n        class_name: GlorotUniform\n        config:\n          dtype: float32\n          seed: null\n      kernel_regularizer: null\n      name: dense\n      trainable: true\n      units: 64\n      use_bias: true\n  - class_name: Dense\n    config:\n      activation: softmax\n      activity_regularizer: null\n      bias_constraint: null\n      bias_initializer:\n        class_name: Zeros\n        config:\n          dtype: float32\n      bias_regularizer: null\n      dtype: float32\n      kernel_constraint: null\n      kernel_initializ

In [16]:
fresh_model = K.models.model_from_yaml(yaml_string)

/Users/rosen/miniconda3/lib/python3.6/site-packages/tensorflow/python/keras/engine/saving.py:345: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(yaml_string)


**注意：** 自定义的模型不能序列化

>Caution: Subclassed models are not serializable because their architecture is defined by the Python code in the body of the call method.
> https://tensorflow.google.cn/guide/keras

## 保存整个模型

keras也可以保留整个模型，包括权重、配置、甚至优化过程，你可以在其他地方加载这个模型继续训练

In [37]:
data = np.random.random((1000, 32))
label = np.random.random((1000, 10))

model.fit(data, label, batch_size=32, epochs=5)

Epoch 1/5
1000/1000 [==============================] - 0s 181us/sample - loss: 11.5661 - acc: 0.0910
Epoch 2/5
1000/1000 [==============================] - 0s 39us/sample - loss: 11.5649 - acc: 0.1040
Epoch 3/5
1000/1000 [==============================] - 0s 40us/sample - loss: 11.5645 - acc: 0.1070
Epoch 4/5
1000/1000 [==============================] - 0s 38us/sample - loss: 11.5642 - acc: 0.1070
Epoch 5/5
1000/1000 [==============================] - 0s 40us/sample - loss: 11.5641 - acc: 0.1050


In [38]:
model.save('./models/my_models.h5')

In [39]:
model = K.models.load_model('./models/my_models.h5')